In [1]:
from lib.KDTreeEncoding import *

import xgboost as xgb
from lib.XGBHelper import *
from lib.XGBoost_params import *
from lib.score_analysis import *

from lib.logger import logger

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
from numpy import load
from glob import glob
import pandas as pd
import pickle as pkl

poverty_dir = '/datasets/cs255-sp22-a00-public/poverty/'
image_dir=poverty_dir+'anon_images/'

In [ ]:
import pickle as pkl
pkl_file='data/Checkpoint.pk'
D=pkl.load(open(pkl_file,'rb'))

for k in D:
    globals()[k]=D[k]
scaling_mean=mean
scaling_std=std

bst_list=[x['bst'] for x in styled_logs[-1]['log']]

In [2]:
# Loading the model we trained for urban images only
pkl_file='data/Checkpoint_Urban.pk'
D=pkl.load(open(pkl_file,'rb'))

for k in D:
    globals()[k]=D[k]
    
scaling_mean_urban = mean
scaling_std_urban = std

bst_urban=[x['bst'] for x in styled_logs[-1]['log']]

In [3]:
# Loading the model we trained for rural images only
pkl_file='data/Checkpoint_Rural.pk'
D=pkl.load(open(pkl_file,'rb'))

for k in D:
    globals()[k] = D[k]
    
scaling_mean_rural = mean
scaling_std_rural = std

bst_rural = [x['bst'] for x in styled_logs[-1]['log']]

In [ ]:

#folds=[{'in':'country_test_reduct.csv','out':'results_country.csv'},
#      {'in':'random_test_reduct.csv','out':'results.csv'}]

folds=[{'in':'test1_test.csv','out':'test1_result.csv'},
      {'in':'test3_test.csv','out':'test3_result.csv'}]


for fold_i in range(len(folds)):
    fold=folds[fold_i]

    #test_csv=f'../public_tables/{fold["in"]}'
    test_csv=f'{fold["in"]}'
    test=pd.read_csv(test_csv)

    out=pd.DataFrame()
    out['filename'] = test['filename']
    out['country']=test['country']
    out['nl_mean']=test['nl_mean']
    out['pred_wo_abstention']=0
    out.set_index('filename', inplace=True)

    # note the tree depth has been set here, it was not there in original code
    Enc_data=encoded_dataset(image_dir,out,tree,depth=8,label_col='pred_wo_abstention')

    data=to_DMatrix(Enc_data.data)
    Preds=zeros([Enc_data.data.shape[0],len(bst_list)])
    for i in range(len(bst_list)):
        Preds[:,i]=bst_list[i].predict(data,output_margin=True)
    Preds=(Preds-scaling_mean)/scaling_std # apply overall score scaling

    _mean=np.mean(Preds,axis=1)
    _std=np.std(Preds,axis=1)
    
    pred_wo_abstention=(2*(_mean>0))-1
    pred_with_abstention=copy(pred_wo_abstention)
    # we are yet to understand how to exploit the thresholds
    pred_with_abstention[(1.25*_std)>abs(_mean)]=0

    out['pred_with_abstention'] = pred_with_abstention
    out['pred_wo_abstention'] = pred_wo_abstention

    outFile=fold["out"]
    out.to_csv(outFile)
    print('\n\n'+'-'*60)
    print(outFile)

In [16]:
# trying changes to the code using separate models for rural and urban 
folds=[{'in':'country_test_reduct.csv','out':'results_country.csv'},
      {'in':'random_test_reduct.csv','out':'results.csv'}]

for fold_i in range(len(folds)):
    fold=folds[fold_i]

    test_csv=f'../public_tables/{fold["in"]}'
    test=pd.read_csv(test_csv)

    out=pd.DataFrame()
    out['filename'] = test['filename']
    out['country']=test['country']
    out['nl_mean']=test['nl_mean']
    out['pred_wo_abstention']=0
    out.set_index('filename', inplace=True)

    Enc_data=encoded_dataset(image_dir,out,tree,depth=8,label_col='pred_wo_abstention')
    data=to_DMatrix(Enc_data.data)
    
    # all predictions for urban
    Preds_urban=zeros([Enc_data.data.shape[0],len(bst_urban)])
    for i in range(len(bst_urban)):
        Preds_urban[:,i] = bst_urban[i].predict(data,output_margin=True)
    Preds_urban = (Preds_urban-scaling_mean_urban)/scaling_std_urban # apply overall score scaling

    _mean_urban = np.mean(Preds_urban,axis=1)
    _std_urban = np.std(Preds_urban,axis=1)
    
    # all predictions for rural
    Preds_rural=zeros([Enc_data.data.shape[0],len(bst_rural)])
    for i in range(len(bst_rural)):
        Preds_rural[:,i] = bst_rural[i].predict(data,output_margin=True)
    Preds_rural = (Preds_rural-scaling_mean_rural)/scaling_std_rural # apply overall score scaling

    _mean_rural = np.mean(Preds_rural,axis=1)
    _std_rural = np.std(Preds_rural,axis=1)
    
    # combining both predictions based on urban/rural status
    preds_wo_abstention = zeros([Enc_data.data.shape[0],1])
    preds_with_abstention = zeros([Enc_data.data.shape[0],1])
    for i, row in test.iterrows():
        if row['urban']:
            preds_wo_abstention[i] = (2*(_mean_urban[i]>0))-1
            preds_with_abstention[i] = 0 if (1.25*_std_urban[i])>abs(_mean_urban[i]) else preds_wo_abstention[i] 
        else:
            preds_wo_abstention[i] = (2*(_mean_rural[i]>0))-1
            preds_with_abstention[i] = 0 if (1.25*_std_rural[i])>abs(_mean_rural[i]) else preds_wo_abstention[i] 
    
    
    out['pred_with_abstention'] = preds_with_abstention
    out['pred_wo_abstention'] = preds_wo_abstention


    outFile=f'data/{fold["out"]}'
    out.to_csv(outFile)
    print('\n\n'+'-'*60)
    print(outFile)

4509 image60.npznpz

------------------------------------------------------------
data/results_country.csv
3779 image13692.npz

------------------------------------------------------------
data/results.csv


In [5]:
# trying changes to the code using separate models for rural and urban 
# with custom test sets
folds=[{'in':'test1_test.csv','out':'test1_result.csv'},
      {'in':'test3_test.csv','out':'test3_result.csv'}]

for fold_i in range(len(folds)):
    fold=folds[fold_i]

    test_csv=f'{fold["in"]}'
    test=pd.read_csv(test_csv)

    out=pd.DataFrame()
    out['filename'] = test['filename']
    out['country']=test['country']
    out['nl_mean']=test['nl_mean']
    out['pred_wo_abstention']=0
    out.set_index('filename', inplace=True)

    Enc_data=encoded_dataset(image_dir,out,tree,depth=8,label_col='pred_wo_abstention')
    data=to_DMatrix(Enc_data.data)
    
    # all predictions for urban
    Preds_urban=zeros([Enc_data.data.shape[0],len(bst_urban)])
    for i in range(len(bst_urban)):
        Preds_urban[:,i] = bst_urban[i].predict(data,output_margin=True)
    Preds_urban = (Preds_urban-scaling_mean_urban)/scaling_std_urban # apply overall score scaling

    _mean_urban = np.mean(Preds_urban,axis=1)
    _std_urban = np.std(Preds_urban,axis=1)
    
    # all predictions for rural
    Preds_rural=zeros([Enc_data.data.shape[0],len(bst_rural)])
    for i in range(len(bst_rural)):
        Preds_rural[:,i] = bst_rural[i].predict(data,output_margin=True)
    Preds_rural = (Preds_rural-scaling_mean_rural)/scaling_std_rural # apply overall score scaling

    _mean_rural = np.mean(Preds_rural,axis=1)
    _std_rural = np.std(Preds_rural,axis=1)
    
    # combining both predictions based on urban/rural status
    preds_wo_abstention = zeros([Enc_data.data.shape[0],1])
    preds_with_abstention = zeros([Enc_data.data.shape[0],1])
    for i, row in test.iterrows():
        if row['urban']:
            preds_wo_abstention[i] = (2*(_mean_urban[i]>0))-1
            preds_with_abstention[i] = 0 if (1.25*_std_urban[i])>abs(_mean_urban[i]) else preds_wo_abstention[i] 
        else:
            preds_wo_abstention[i] = (2*(_mean_rural[i]>0))-1
            preds_with_abstention[i] = 0 if (1.25*_std_rural[i])>abs(_mean_rural[i]) else preds_wo_abstention[i] 
    
    
    out['pred_with_abstention'] = preds_with_abstention
    out['pred_wo_abstention'] = preds_wo_abstention


    outFile=fold["out"]
    out.to_csv(outFile)
    print('\n\n'+'-'*60)
    print(outFile)

999 image16235.npz

------------------------------------------------------------
test1_result.csv
999 image3695.npzz

------------------------------------------------------------
test3_result.csv


In [14]:
df1 = pd.read_csv('test1.csv')
df1["label"] = df1["label"].replace({0:-1})
df2 = pd.read_csv('test1_result.csv')
sym_score = 0
asym_score = 0
for i in range(df1.shape[0]):
    if df1['label'][i] == df2['pred_wo_abstention'][i]:
        sym_score += 1
    if df1['label'][i] == df2['pred_with_abstention'][i]:
        asym_score += 1
    if df1['label'][i] != df2['pred_wo_abstention'][i]:
        sym_score += -2
    if df1['label'][i] != df2['pred_with_abstention'][i] and df2['pred_with_abstention'][i] != 0:
        asym_score += -2
    if df1['label'][i] != df2['pred_with_abstention'][i] and df2['pred_with_abstention'][i] == 0:
        asym_score += 0
        
print("Symmetric Score: ", sym_score)
print("Asymmetric Score: ", asym_score)

Symmetric Score:  451
Asymmetric Score:  474
